# Code 2 - SQL Agents

## Step 1 - Connect to Database

### Import Packages

In [2]:
from sqlite3 import connect
from langchain_community.utilities.sql_database import SQLDatabase

### Create database connection for LLM (create db)

In [3]:
db = SQLDatabase.from_uri("sqlite:///chinook.db", sample_rows_in_table_info = 3)
print(len(db.get_usable_table_names()), db.get_usable_table_names())

11 ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


### Check Connection

In [4]:
con = connect("chinook.db")
cur = con.execute("Select Count(Distinct(AlbumId)) from Album;")
print(cur.fetchall())

[(347,)]


## Step 2 - Setup Langchain SQL Chain

### Import API Key

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
API_KEY = os.getenv("GROQ_API_KEY")

### Import Packages

In [7]:
from langchain.callbacks import get_openai_callback
from langchain_groq import ChatGroq

### Create an Instance of LLM with configuration & Check (creating llm)

In [23]:
llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0)  #mixtral-8x7b-32768 | llama-3.1-8b-instant | llama3-8b-8192
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000028841EB7910>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000028841F42E10>, temperature=1e-08, groq_api_key=SecretStr('**********'))

## Step 3 Setup Agent

Advantages of SQL Agents:

1. Saves tokens - Retrieving only required schema & relevant tables.
2. Recover from errors by generating queries, catch the traceback and regenerate correctly
3. Chains run on a predefined pattern; Agents is defined by llm; The LLM then chooses the right tools to execute

### Import Packages

In [24]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType

### Create an Instance of Agent with llm and db

In [25]:
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True, stream_runnable=False)

### Execute LLM Agent

In [26]:
result = agent_executor.invoke("Which country's customers spent the most?")
print(result)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Invoice'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lim

### Execute LLM Agent with Callback

In [27]:
with get_openai_callback() as cb:
    result = agent_executor.invoke("Which country's customers spent the most?")
print("LLM Result: ", result['output'])
print(cb)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Customer, Invoice'}`



CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa Brasileira de Aeronáutica S.A.	Av. Brigadeiro Faria Lim

BadRequestError: Error code: 400 - {'error': {'message': "Failed to call a function. Please adjust your prompt. See 'failed_generation' for more details.", 'type': 'invalid_request_error', 'code': 'tool_use_failed', 'failed_generation': 'Now I can construct a query to find out which country\'s customers spent the most.  I will need to join the Invoice and Customer tables, group by Country, and sum the Total column.  I will also need to order by the sum of Total in descending order.\n\n<tool-use>{"tool\\_call":{"id":"call\\_8d9s","type":"function","function":{"name":"sql\\_db\\_query"},"parameters":{"query":"SELECT Customer.Country, SUM(Invoice.Total) FROM Invoice JOIN Customer ON Invoice.CustomerId = Customer.CustomerId GROUP BY Customer.Country ORDER BY SUM(Invoice.Total) DESC;"}}}</tool-use>'}}

### Check Execution Flow

In [ ]:
print(agent_executor)

In [ ]:
print(agent_executor.agent.runnable.get_prompts()[0].messages[0].content)

In [ ]:
agent_executor.agent.runnable.to_json()['kwargs']['middle'][1].kwargs['tools']

# END - Next Topic React SQL Agents